# Brief Introduction to CANTERA

For additional detail, see [online documentation](http://www.cantera.org/docs/sphinx/html/index.html)

## Thermodynamic Properties

Examples below both follow the first part of the lecture, but also introduce some basic features of `Cantera`.

In [1]:
# import relevant python modules
import cantera as ct

In [2]:
# some constants
ct.one_atm, ct.gas_constant,

(101325.0, 8314.4621)

Note that jupyter supports auto-completion, i.e. if you start typing and hit `TAB`, a list of possible completions appears. E.g. if you start typing `ct.g` and hit `TAB`, you'll see `ct.gas_constant` and `ct.get_data_directories`.

In [3]:
# please read above, move the cursor to the end of `ct.g` below and hit TAB ...
ct.g

AttributeError: module 'cantera' has no attribute 'g'

#### Gas Mixtures

In [4]:
# create a mixture that contains species typical for air (some property data already ship with cantera)
gas = ct.Solution('air.xml')
gas.species_names

['O', 'O2', 'N', 'NO', 'NO2', 'N2O', 'N2', 'AR']

In [5]:
# number of species
gas.n_species

8

In [6]:
# elements making up mixture
gas.element_names

['O', 'N', 'Ar']

In [7]:
# list molecular weights
gas.molecular_weights

array([ 15.9994 ,  31.9988 ,  14.00674,  30.00614,  46.00554,  44.01288,
        28.01348,  39.948  ])

In [8]:
# gas temperature, pressure
gas.T, gas.P,

(300.0, 101325.0)

In [9]:
# shorthand for the above
gas.TP

(300.0, 101325.0)

In [10]:
# gas composition (defaults to air): mole basis
gas.X

array([ 0.  ,  0.21,  0.  ,  0.  ,  0.  ,  0.  ,  0.78,  0.01])

In [11]:
# entries are indexed according to the list of species names
gas.species_index('O2'), gas.species_name(1), gas.X[1]

(1, 'O2', 0.21000000000000002)

In [12]:
# gas composition (mass basis)
gas.Y

array([ 0.        ,  0.23195746,  0.        ,  0.        ,  0.        ,
        0.        ,  0.75425298,  0.01378956])

In [13]:
# arrays seen above are numpy arrays, which support shortcuts (please note that the small
# difference - 2.e-16 - is caused by machine precision, i.e. it is not real)
gas.X.sum(),

(1.0000000000000002,)

**Note:** Python's `numpy` arrays by default apply element-wise multiplications. 

Consider mixture properties, - e.g. the average molecular weight $MW_\mathrm{avg}$, - which are obtained as a weighed sum:

\begin{equation}
MW_\mathrm{avg} = \sum_{i=0}^{N-1} X_i \, MW_i \neq \frac{1}{N} \sum_{i=0}^{N-1} MW_i
\end{equation}

with $N$ being the number of species:

In [14]:
# comparison of built-in and manually coded results
gas.mean_molecular_weight, (gas.X*gas.molecular_weights).sum(), gas.molecular_weights.mean()

(28.9697424, 28.969742400000005, 31.248872499999997)

In [15]:
# another shorthand describing the complete mixture state
gas.TPX

(300.0,
 101325.0,
 array([ 0.  ,  0.21,  0.  ,  0.  ,  0.  ,  0.  ,  0.78,  0.01]))

#### Finding gas properties (`attributes` of gas mixtures)

Note: apart from properties, attributes also include functions defined for the gas mixture.

In [16]:
# the long way (note that the `print` command creates a more compact view of a relatively long list)
print(dir(gas))

['DP', 'DPX', 'DPY', 'HP', 'HPX', 'HPY', 'ID', 'P', 'P_sat', 'SP', 'SPX', 'SPY', 'SV', 'SVX', 'SVY', 'T', 'TD', 'TDX', 'TDY', 'TP', 'TPX', 'TPY', 'T_sat', 'UV', 'UVX', 'UVY', 'X', 'Y', '__call__', '__class__', '__copy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_check_kinetics_species_index', '_check_phase_index', '_check_reaction_index', '_full_states', '_init_cti_xml', '_init_parts', '_references', 'add_reaction', 'add_species', 'atomic_weight', 'atomic_weights', 'basis', 'binary_diff_coeffs', 'chemical_potentials', 'concentrations', 'cp', 'cp_mass', 'cp_mole', 'creation_rates', 'critical_density', 'critical_pressure', 'critical_temperature', 'cv', 'cv_mass', 'cv_mole', 'delta_e

In [17]:
# some python tricks help here (this is a shortcut for a loop combined with an if statement)
# doesn't really have an equivalent in matlab ;)
[ attr for attr in dir(gas) if 'enthalpy' in attr ]

['delta_enthalpy', 'delta_standard_enthalpy', 'enthalpy_mass', 'enthalpy_mole']

In [18]:
# same as above, only longer
keep = []
# loop over everything that can be retrieved for the gas mixture
for attr in dir(gas):
    # check whether the string 'enthalpy' is part of the attribute
    if 'enthalpy' in attr: 
        keep.append(attr)
# display result
keep

['delta_enthalpy', 'delta_standard_enthalpy', 'enthalpy_mass', 'enthalpy_mole']

In [19]:
# enthalpies (note that `h` is also present)
gas.enthalpy_mole, gas.enthalpy_mass, gas.h, 

(54867.91309526664, 1893.97310951811, 1893.97310951811)

In [20]:
# commands for specific heats
[ attr for attr in dir(gas) if 'cp' in attr ]

['cp', 'cp_mass', 'cp_mole', 'partial_molar_cp', 'standard_cp_R']

In [21]:
# values default to mass basis
gas.cp_mole, gas.cp_mass, gas.cp, 

(29058.230869362633, 1003.0545135038077, 1003.0545135038078)

#### Changing Mixture States

In [22]:
# note that it requires *two* properties to define a state
gas.TP = 1200, ct.one_atm,
gas.TP

(1200.0, 101325.0)

In [23]:
# changing the mixture can be done using strings
gas.X = 'O2:1'
gas.X

array([ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.])

In [24]:
# or
gas.X = 'O2:.5,N2:.5'
gas.X

array([ 0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ])

**Important:** all mixture properties are defined assuming fractions add up to unity, i.e.

\begin{equation}
\sum_{i=0}^{N-1} X_i = 1
\end{equation} 

and 

\begin{equation}
\sum_{i=0}^{N-1} Y_i = 1
\end{equation}

with $N$ being the number of species.

In [25]:
# i.e. cantera will normalize the following
gas.X = 'O2:1,N2:1'
gas.X

array([ 0. ,  0.5,  0. ,  0. ,  0. ,  0. ,  0.5,  0. ])

In [26]:
# i.e. whatever mixture property you may want to evaluate is obtained for one kmol (or one kg) of mixture
(gas.int_energy_mole * gas.X).sum(), (gas.int_energy_mass * gas.Y).sum()

(18963956.705439322, 632002.54032805702)

In [27]:
# you can show that there is some small error in how thermodynamic properties are internally tabulated
# Note: what is the theoretical value? And: how bad is this error?
gas.TP = 298.15, ct.one_atm
(gas.enthalpy_mole * gas.X).sum(), (gas.enthalpy_mass * gas.Y).sum()

(714.95915118031883, 23.827095093881415)

In [28]:
# we could go further and look at elemental species
gas.element_names

['O', 'N', 'Ar']

In [29]:
# and
[ gas.elemental_mole_fraction(a) for a in gas.element_names ]

[0.5, 0.5, 0.0]

In [30]:
# or
[ gas.elemental_mass_fraction(a) for a in gas.element_names ]

[0.5332042042062058, 0.46679579579379427, 0.0]